In [14]:
import plotly.graph_objects as go
import plotly.express as px
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yaml
import plotly.express as px
import gc
import re
import os


In [2]:
import sys
sys.path.append("../../scripts/pypsa-de")

In [4]:
# Tech Colors aus AriadneReport laden
with open("../config/plotting.default.yaml", "r") as f:
    plotting_config = yaml.safe_load(f)

tech_colors = plotting_config["plotting"]["tech_colors"]

In [5]:
# ====================
# 2. Vor-Definitionen
# ====================

# 2.1 Farben und Linienstile für BEV-Charger-Plot
bev_colors = {
    "EV battery (BEV charger) - Min":   "violet",
    "EV battery (BEV charger) - Mean":  "green",
    "EV battery (BEV charger) - Max":   "blue"
}
bev_dash = {
    "EV battery (BEV charger) - Min":   "dot",
    "EV battery (BEV charger) - Mean":  "solid",
    "EV battery (BEV charger) - Max":   "dot"
}

# 2.2 Mapping der Carrier (für Discharge-Plot)
carrier_map = {
    "battery discharger":        "Utility-scale battery",
    "home battery discharger":   "Home battery",
    "V2G":                       "EV battery"
}

# 2.3 Speichertyp-Konfiguration (für extract_metrics)
storage_config = {
    "Utility-scale battery": {"store": "battery",           "link": "battery discharger"},
    "Home battery":           {"store": "home battery",     "link": "home battery discharger"},
    "EV battery":             {"store": "EV battery",       "link": "BEV charger"}
}

# 2.4 Hilfsfunktion: Versucht, aus einem Dateinamen eine vierstellige Jahreszahl zu extrahieren
def parse_year_from_filename(filename: str) -> int:
    m = re.search(r"(\d{4})", filename)
    return int(m.group(1)) if m else None

# 2.5 Funktion: BEV-Charger (Min/Mean/Max) in Deutschland aus einem Netzwerk extrahieren (in GW)
def get_bev_charger_range_DE(net: pypsa.Network):
    bev_idx = net.links.index[
        (net.links.carrier == "BEV charger") &
        (net.links.bus0.str.startswith("DE"))
    ]
    if len(bev_idx) == 0:
        return 0.0, 0.0, 0.0

    eff_p = net.links_t.p_max_pu[bev_idx].multiply(net.links.p_nom[bev_idx], axis=1)
    summed = eff_p.sum(axis=1)

    p_min  = summed.min()  / 1e3  # MW → GW
    p_mean = summed.mean() / 1e3
    p_max  = summed.max()  / 1e3
    return p_min, p_mean, p_max


In [6]:
####### definitions aus Ariadne Report #######
plt.rcParams["font.family"] = "DejaVu Sans"
extent_de = [5.5, 15.5, 47, 56]

THRESHOLD = 5  # GW

CARRIER_GROUPS = {
    "electricity": ["AC", "low voltage"],
    "heat": [
        "urban central heat",
        "urban decentral heat",
        "rural heat",
        "residential urban decentral heat",
        "residential rural heat",
        "services urban decentral heat",
        "services rural heat",
    ],
    # "hydrogen": "H2",
    # "oil": "oil",
    # "methanol": "methanol",
    # "ammonia": "NH3",
    # "biomass": ["solid biomass", "biogas"],
    # "CO2 atmosphere": "co2",
    # "CO2 stored": "co2 stored",
    # "methane": "gas",
}

backup_techs = {
    "Gas": [
        "OCGT",
        "CCGT",
        "biogas",
        "urban central gas CHP",
        "urban central gas CHP CC",
    ],
    # "Öl": ["oil", "urban central oil CHP"],
    "Kohle": ["lignite", "coal", "urban central coal CHP", "urban central lignite CHP"],
    "Wasserkraft": ["PHS", "hydro"],
    "Batterie": ["battery discharger", "home battery discharger"],
    "Wasserstoff": [
        "H2 OCGT",
        "H2 retrofit OCGT",
        "urban central H2 CHP",
        "urban central H2 retrofit CHP",
    ],
    # "Müllverbrennung": ["waste CHP", "waste CHP CC"],
    "Biomasse": [
        "solid biomass",
        "urban central solid biomass CHP",
        "urban central solid biomass CHP CC",
    ],
}

vre_gens = [
    "onwind",
    "offwind-ac",
    "offwind-dc",
    "solar",
    "solar-hsat",
    "solar rooftop",
    "ror",
]

year_colors = [
    "dimgrey",
    "darkorange",
    "seagreen",
    "cadetblue",
    "hotpink",
    "darkviolet",
    "gold",
]
markers = [
    "v",
    "^",
    "<",
    ">",
    "1",
    "2",
    "3",
    "4",
    "*",
    "+",
    "d",
    "o",
    "|",
    "s",
    "P",
    "p",
    "h",
]

date_format = "%Y-%m-%d %H:%M:%S"
reduced_date_format = "%Y-%m-%d"


resistive_heater = [
    "urban central resistive heater",
    "rural resistive heater",
    "urban decentral resistive heater",
]
gas_boiler = [
    "urban central gas boiler",
    "rural gas boiler",
    "urban decentral gas boiler",
]
air_heat_pump = [
    "urban central air heat pump",
    "rural air heat pump",
    "urban decentral air heat pump",
]
water_tanks_charger = [
    "urban central water tanks charger",
    "rural water tanks charger",
    "urban decentral water tanks charger",
]
water_tanks_discharger = [
    "urban central water tanks discharger",
    "rural water tanks discharger",
    "urban decentral water tanks discharger",
]
solar_thermal = [
    "urban decentral solar thermal",
    "urban central solar thermal",
    "rural solar thermal",
]
carrier_renaming = {
    "urban central solid biomass CHP CC": "biomass CHP CC",
    "urban central solid biomass CHP": "biomass CHP",
    "urban central gas CHP": "gas CHP",
    "urban central gas CHP CC": "gas CHP CC",
    "urban central air heat pump": "air heat pump",
    "urban central resistive heater": "resistive heater",
}
carrier_renaming_reverse = {
    "biomass CHP CC": "urban central solid biomass CHP CC",
    "biomass CHP": "urban central solid biomass CHP",
    "gas CHP": "urban central gas CHP",
    "gas CHP CC": "urban central gas CHP CC",
    "air heat pump": "urban central air heat pump",
    "resistive heater": "urban central resistive heater",
}
c1_groups = [
    resistive_heater,
    gas_boiler,
    air_heat_pump,
    water_tanks_charger,
    water_tanks_discharger,
    solar_thermal,
]
c1_groups_name = [
    "resistive heater",
    "gas boiler",
    "air heat pump",
    "water tanks charger",
    "water tanks discharger",
    "solar thermal",
]
solar = [
    "solar rooftop",
    "solar-hsat",
    "solar",
]
electricity_load = [
    "electricity",
    "industry electricity",
    "agriculture electricity",
]
electricity_imports = [
    "AC",
    "DC",
]

carriers_in_german = {
    "DC": "Gleichstrom",
    "AC": "Wechselstrom",
    "hydro": "Wasserkraft (Reservoir & Damm)",
    "offwind-ac": "Offshore-Wind (AC)",
    "offwind-dc": "Offshore-Wind (DC)",
    "solar": "Solar",
    "solar-hsat": "Solar (HSAT)",
    "onwind": "Onshore-Wind",
    "PHS": "Pumpspeicherkraftwerk",
    "ror": "Laufwasserkraft",
    "": "",
    "lignite": "Braunkohle",
    "coal": "Steinkohle",
    "oil": "Öl",
    "uranium": "Uran",
    "none": "keine",
    "co2": "CO2",
    "co2 stored": "CO2 gespeichert",
    "co2 sequestered": "CO2 sequestriert",
    "gas": "Gas",
    "H2": "Wasserstoffspeicher",
    "battery": "Batteriespeicher",
    "EV battery": "Elektrofahrzeug-Batterie",
    "urban central heat": "Zentrale städtische Heizung",
    "urban central water tanks": "Zentrale städtische Wassertanks",
    "urban central solar thermal": "Zentrale städtische Solarthermie",
    "biogas": "Biogas",
    "solid biomass": "Biomasse",
    "methanol": "Methanol",
    "home battery": "Hausbatterie",
    "rural heat": "Ländliche Wärme",
    "rural solar thermal": "Ländliche Solarthermie",
    "rural water tanks": "Ländliche Wassertanks",
    "urban decentral heat": "Dezentrale städtische Heizung",
    "urban decentral solar thermal": "Dezentrale städtische Solarthermie",
    "urban decentral water tanks": "Dezentrale städtische Wassertanks",
    "oil primary": "Primäröl",
    "solar rooftop": "Solar-Dach",
    "urban central heat vent": "Zentrale städtische Wärmeentlüftung",
    "gas primary": "Primärgas",
    "solid biomass for industry": "Biomasse (Industrie)",
    "shipping oil": "Schiffsöl",
    "agriculture machinery oil": "Landwirtschaftsmaschinenöl",
    "naphtha for industry": "Naphtha für die Industrie",
    "land transport oil": " Öl (Transport)",
    "kerosene for aviation": "Kerosin für die Luftfahrt",
    "non-sequestered HVC": "Nicht-sequestriertes HVC",
    "shipping methanol": "Methanol für Schifffahrt",
    "gas for industry": "Gas (Industrie)",
    "low voltage": "Niederspannung",
    "industry methanol": "Methanol (Industrie)",
    "coal for industry": "Kohle (Industrie)",
    "process emissions": "Prozessemissionen",
    "industry electricity": "Industrieelektrizität",
    "low-temperature heat for industry": "Niedertemperaturwärme (Industrie)",
    "agriculture electricity": "Landwirtschaftliche Elektrizität",
    "electricity": "Elektrizität",
    "land transport EV": "Elektrofahrzeuge (Transport)",
    "agriculture heat": "Landwirtschaftliche Wärme",
    "urban central air heat pump": "Zentrale städtische Luftwärmepumpe",
    "electricity distribution grid": "Stromverteilungsnetz",
    "battery charger": "Batterie Laden",
    "waste CHP": "Müll-KWK",
    "urban central water tanks discharger": "Entladung zentraler städtischer Wassertanks",
    "rural water tanks discharger": "Entladung ländlicher Wassertanks",
    "urban decentral biomass boiler": "Dezentrale städtische Biomassekessel",
    "BEV charger": "E-Fahrzeug Laden",
    "rural ground heat pump": "Erdwärmepumpe",
    "Fischer-Tropsch": "Fischer-Tropsch",
    "urban decentral water tanks discharger": "Entladung dezentraler städtischer Wassertanks",
    "urban central gas CHP": "Gas-KWK",
    "Sabatier": "Sabatier-Prozess",
    "gas compressing": "Gasverdichtung",
    "home battery charger": "Hausbatterie Laden",
    "battery discharger": "Batterie Entladung",
    "H2 pipeline retrofitted": "Nachgerüstete Wasserstoffpipeline",
    "rural resistive heater": "Ländlicher Widerstandsheizer",
    "urban central water tanks charger": "Ladung zentraler städtischer Wassertanks",
    "urban central solid biomass CHP CC": "Biomasse-KWK mit CO2-Abscheidung",
    "rural air heat pump": "Ländliche Luftwärmepumpe",
    "DAC": "Direkte CO2-Abscheidung",
    "urban decentral air heat pump": "Dezentrale städtische Luftwärmepumpe",
    "waste CHP CC": "Müll-KWK mit CO2-Abscheidung",
    "biomass to liquid CC": "Biomasse zu Flüssigkeit mit CO2-Abscheidung",
    "HVC to air": "HVC in die Luft",
    "methanolisation": "Methanolisation",
    "gas for industry CC": "Gas mit CO2-Abscheidung (Industrie)",
    "H2 pipeline": "Wasserstoffpipeline",
    "solid biomass for industry CC": "Biomasse mit CO2-Abscheidung (Industrie)",
    "urban decentral gas boiler": "Dezentrale städtische Gaskessel",
    "urban central gas CHP CC": "Gas-KWK mit CO2-Abscheidung",
    "rural gas boiler": "Ländlicher Gaskessel",
    "process emissions CC": "Prozessemissionen mit CO2-Abscheidung",
    "urban decentral water tanks charger": "Ladung dezentraler städtischer Wassertanks",
    "biogas to gas CC": "Biogas zu Gas mit CO2-Abscheidung",
    "urban decentral resistive heater": "Dezentrale städtische Widerstandsheizer",
    "biogas to gas": "Biogas zu Gas",
    "OCGT": "Gas (OCGT)",
    "urban central solid biomass CHP": "Biomasse-KWK",
    "urban central gas boiler": "Zentraler städtischer Gaskessel",
    "urban central resistive heater": "Zentraler städtischer Widerstandsheizer",
    "oil refining": "Ölraffinierung",
    "rural biomass boiler": "Ländlicher Biomassekessel",
    "SMR": "Dampfreformierung",
    "biomass to liquid": "Biomasse zu Flüssigkeit",
    "rural water tanks charger": "Ladung ländlicher Wassertanks",
    "home battery discharger": "Hausbatterie Entladung",
    "H2 Store": "Wasserstoffspeicher",
    "renewable oil": "Erneuerbares Öl",
    "renewable gas": "Erneuerbares Gas",
    "CCGT": "Gas (CCGT)",
    "nuclear": "Kernenergie",
    "gas CHP": "Gas-KWK",
    "gas CHP CC": "Gas KWK mit CO2-Abscheidung",
    "urban central coal CHP": "Steinkohle-KWK",
    "Electricity trade": "Stromhandel",
    "urban central biomass CHP": "Biomasse-KWK",
    "biomass CHP": "Biomasse-KWK",
    "air heat pump": "Luftwärmepumpe",
    "Electricity load": "Stromlast",
    "resistive heater": "Widerstandsheizung",
    "gas boiler": "Gaskessel",
    "H2 Electrolysis": "Elektrolyse",
    "H2 Fuel Cell": "Brennstoffzelle (Strom)",
    "H2 for industry": "H2 für Industrie",
    "H2 OCGT": "Wasserstoff (OCGT)",
    "H2 CHP": "H2 KWK",
    "land transport fuel cell": "Brennstoffzelle (Verkehr)",
    "other": "Sonstige",
    "SMR CC": "Dampfreformierung mit CCS",
    "H2 pipeline (new)": "H2 Pipeline (Neubau)",
    "H2 pipeline (repurposed)": "H2 Pipeline (Umstellung)",
    "H2 pipeline (Kernnetz)": "H2 Pipeline (Kernnetz)",
    "heat pump": "Wärmepumpe",
    "urban central H2 CHP": "Wasserstoff-KWK",
    "urban central H2 retrofit CHP": "Wasserstoff-KWK (Umrüstung)",
    "urban central oil CHP": "Öl-KWK",
    "urban central lignite CHP": "Braunkohle-KWK",
    "H2 retrofit OCGT": "Wasserstoff (OCGT;Umrüstung)",
}


tech_colors.update({
    "H2 OCGT": "#e78ac3",
    "H2 retrofit OCGT": "#fbb4ae",
    "urban central H2 CHP": "#fccde5",
    "urban central H2 retrofit CHP": "#d9d9d9"
})


In [7]:
# ============================================
# 3. Funktion: Daten aus EINEM Netzwerk extrahieren
# ============================================

def extract_metrics_from_network(net: pypsa.Network, scenario: str, year_label) -> dict:
    """
    Extrahiert aus net (pypsa.Network) folgende Kenngrößen:
      - energy_rows:   Speicherkapazität (e_nom_opt) in GWh pro Speichertyp (DE-only)
      - power_rows:    Entladeleistung (p_nom_opt) in GW pro Speichertyp (DE-only)
      - discharge_rows: Jährliche Strombereitstellung (statistics.supply) in TWh pro Carrier (DE-only)
      - bev_rows:      BEV‐Charger‐Range (Min/Mean/Max) in GW
    
    Gibt ein Dictionary zurück:
      {
        "energy":    [ {...}, {...}, ... ],
        "power":     [ {...}, {...}, ... ],
        "discharge": [ {...}, {...}, ... ],
        "bev":       [ {...}, {...}, {...} ]  # genau drei Einträge pro Net (Min/Mean/Max)
      }
    """
    energy_rows    = []
    power_rows     = []
    discharge_rows = []
    bev_rows       = []

    # 1) Kapazität (e_nom_opt) & Leistung (p_nom_opt) pro Speichertyp
    for label, ids in storage_config.items():
        store_idx = net.stores.index[
            (net.stores.carrier == ids["store"]) &
            (net.stores.bus.str.startswith("DE"))
        ]
        e_nom = net.stores.e_nom_opt[store_idx].sum() / 1e3 if len(store_idx) > 0 else 0.0

        link_idx = net.links.index[
            (net.links.carrier == ids["link"]) &
            (net.links.bus0.str.startswith("DE"))
        ]
        p_nom = net.links.p_nom_opt[link_idx].sum() / 1e3 if len(link_idx) > 0 else 0.0

        energy_rows.append({
            "Scenario":     scenario,
            "Year":         year_label,
            "Type":         label,
            "Value [GWh]":  e_nom
        })
        power_rows.append({
            "Scenario":     scenario,
            "Year":         year_label,
            "Type":         label,
            "Value [GW]":   p_nom
        })

    # 2) Discharge-Statistik (statistics.supply) in TWh
    df_supply = (
        net.statistics.supply(
            bus_carrier=["low voltage", "AC"],
            groupby=["name", "bus", "carrier"],
            nice_names=False
        )
        .filter(like="DE", axis=0)
        .groupby("carrier").sum()
        .drop(["AC", "DC", "electricity distribution grid"], errors="ignore")
        / 1e6  # MWh → TWh
    )
    for carrier, value in df_supply.items():
        if carrier in ["battery discharger", "home battery discharger", "V2G"]:
            discharge_rows.append({
                "Scenario":    scenario,
                "Year":        year_label,
                "Type":        carrier,
                "Value [TWh]": value
            })

    # 3) BEV‐Charger‐Range (Min/Mean/Max) in GW
    p_min, p_mean, p_max = get_bev_charger_range_DE(net)
    bev_rows.append({
        "Scenario":   scenario,
        "Year":       year_label,
        "Type":       "EV battery (BEV charger) - Min",
        "Value [GW]": p_min
    })
    bev_rows.append({
        "Scenario":   scenario,
        "Year":       year_label,
        "Type":       "EV battery (BEV charger) - Mean",
        "Value [GW]": p_mean
    })
    bev_rows.append({
        "Scenario":   scenario,
        "Year":       year_label,
        "Type":       "EV battery (BEV charger) - Max",
        "Value [GW]": p_max
    })

    return {
        "energy":    energy_rows,
        "power":     power_rows,
        "discharge": discharge_rows,
        "bev":       bev_rows
    }


In [8]:
# ============================================
# 4. Funktion: Alle Netzwerke eines Base-Folders einlesen,
#    mit optionaler Möglichkeit, einzelne Szenarien auszuschließen
# ============================================

def load_all_scenarios(base_folder: str, exclude_scenarios: list = None) -> dict:
    """
    Liest alle Szenarien in base_folder/*/postnetworks/*.nc ein,
    ruft extract_metrics_from_network() auf und gibt ein Dict zurück:
      {
        "energy":    DataFrame,
        "power":     DataFrame,
        "discharge": DataFrame,
        "bev":       DataFrame
      }
    
    Parameter:
      - base_folder        : Pfad zu deinem Ordner mit den Szenarien-Unterverzeichnissen.
      - exclude_scenarios  : (optional) Liste von Szenario-Namen, die beim Einlesen übersprungen werden sollen.
                            Beispiel: ["FokusStrom", "SzenarioXYZ"]
                            Standardmäßig None (d.h. kein Szenario wird ausgeschlossen).

    Vorgehen:
      1. Ermittele alle Unterordner, die ein Unterverzeichnis "postnetworks" enthalten.
      2. Filtere dabei jene in exclude_scenarios heraus (falls angegeben).
      3. Für jedes verbleibende Szenario werden alle .nc-Dateien in ".../postnetworks" geladen und über
         extract_metrics_from_network() ausgelesen. BEV-Range wird direkt mitgesammelt.
      4. Am Ende werden vier DataFrames zurückgegeben: energy, power, discharge, bev.
    """
    if exclude_scenarios is None:
        exclude_scenarios = []

    all_energy_rows    = []
    all_power_rows     = []
    all_discharge_rows = []
    all_bev_rows       = []

    # Szenarien ermitteln (Ordner wie base_folder/<Szenario>/postnetworks)
    szenarien = [
        d for d in os.listdir(base_folder)
        if os.path.isdir(os.path.join(base_folder, d, "networks"))
    ]

    # Szenarien herausfiltern, die ausgeschlossen werden sollen
    szenarien = [s for s in szenarien if s not in exclude_scenarios]
    print("Szenarien, die eingelesen werden:", szenarien)
    if exclude_scenarios:
        print("Ausgeschlossene Szenarien:", exclude_scenarios)

    for scenario in szenarien:
        net_folder = os.path.join(base_folder, scenario, "networks")
        nc_files = [f for f in os.listdir(net_folder) if f.lower().endswith(".nc")]

        if not nc_files:
            print(f"  → {scenario}: Keine .nc-Dateien gefunden, überspringe.")
            continue

        print(f"\n=== Verarbeite Szenario '{scenario}' ({len(nc_files)} Netzwerke) ===")
        for fn in nc_files:
            pfad = os.path.join(net_folder, fn)
            jahr = parse_year_from_filename(fn)
            year_label = jahr if jahr is not None else fn

            print(f"  • Lade {scenario}/{fn} …", end=" ")
            net = pypsa.Network(pfad)
            print("fertig.")

            metrics = extract_metrics_from_network(net, scenario, year_label)
            all_energy_rows   .extend(metrics["energy"])
            all_power_rows    .extend(metrics["power"])
            all_discharge_rows.extend(metrics["discharge"])
            all_bev_rows      .extend(metrics["bev"])

            del net
            gc.collect()

        print(f"— Fertig mit Szenario '{scenario}'")

    # Aus Listen entstehen DataFrames
    df_energy    = pd.DataFrame(all_energy_rows)
    df_power     = pd.DataFrame(all_power_rows)
    df_discharge = pd.DataFrame(all_discharge_rows)
    df_bev_range = pd.DataFrame(all_bev_rows)

    # Sortierung (optional)
    sort_cols = ["Scenario", "Type", "Year"]
    df_energy    = df_energy   .sort_values(sort_cols).reset_index(drop=True)
    df_power     = df_power    .sort_values(sort_cols).reset_index(drop=True)
    df_discharge = df_discharge.sort_values(sort_cols).reset_index(drop=True)
    df_bev_range = df_bev_range.sort_values(sort_cols).reset_index(drop=True)

    return {
        "energy":    df_energy,
        "power":     df_power,
        "discharge": df_discharge,
        "bev":       df_bev_range
    }


In [10]:
results = load_all_scenarios("results", exclude_scenarios=["KN2045_Mix_Base", "KN2045_Mix_3H_DE_1ZONE_FW", "KN2045_Mix"])
df_energy    = results["energy"]
df_power     = results["power"]
df_discharge = results["discharge"]
df_bev_range = results["bev"]


Szenarien, die eingelesen werden: ['KN2045_Mix_1H_DE_1ZONE', 'KN2045_Mix_3H', 'KN2045_Mix_3H_DE_1ZONE', 'KN2045_Mix_Base_home_battery', 'Technologiemix']
Ausgeschlossene Szenarien: ['KN2045_Mix_Base', 'KN2045_Mix_3H_DE_1ZONE_FW', 'KN2045_Mix']

=== Verarbeite Szenario 'KN2045_Mix_1H_DE_1ZONE' (7 Netzwerke) ===
  • Lade KN2045_Mix_1H_DE_1ZONE/base_s_20_hourly_none_2020.nc … 

INFO:pypsa.io:Imported network base_s_20_hourly_none_2020.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


fertig.
  • Lade KN2045_Mix_1H_DE_1ZONE/base_s_20_hourly_none_2025.nc … 

INFO:pypsa.io:Imported network base_s_20_hourly_none_2025.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


fertig.
  • Lade KN2045_Mix_1H_DE_1ZONE/base_s_20_hourly_none_2030.nc … 

...

fertig.
  • Lade Technologiemix/base_s_49_lvopt__none_2040.nc … 

Index(['FR4 0 waste CHP-2040', 'IT3 0 waste CHP-2040',
       'FR4 0 waste CHP CC-2040', 'IT3 0 waste CHP CC-2040',
       'IT3 0 urban central gas CHP-2005', 'IT3 0 urban central gas CHP-2010'],
      dtype='object', name='name')
INFO:pypsa.io:Imported network base_s_49_lvopt__none_2040.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


fertig.
  • Lade Technologiemix/base_s_49_lvopt__none_2045.nc … 

Index(['FR4 0 waste CHP-2045', 'IT3 0 waste CHP-2045',
       'FR4 0 waste CHP CC-2045', 'IT3 0 waste CHP CC-2045',
       'IT3 0 urban central gas CHP-2010'],
      dtype='object', name='name')
INFO:pypsa.io:Imported network base_s_49_lvopt__none_2045.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


fertig.
— Fertig mit Szenario 'Technologiemix'


In [ ]:
# Einzelne Szenarien nachträglich laden und an bestehende DataFrames anhängen

# 1) Bereits existierende DataFrames (alle alten Szenarien)
df_energy    # <bereits gebaut>
df_power     # <bereits gebaut>
df_discharge # <bereits gebaut>

# 2) Lese nur das fehlende Szenario nach:
base_folder = "results"
new_scenario = "KN2045_Mix_Base_home_battery"

# Wir können load_all_scenarios modifizieren, sodass es wahlweise NUR
# dieses eine Szenario einliest (statt über alle zu iterieren). 
# Dafür definieren wir eine Variante:

def load_single_scenario(base_folder: str, scenario_name: str):
    net_folder = os.path.join(base_folder, scenario_name, "networks")
    nc_files = [f for f in os.listdir(net_folder) if f.lower().endswith(".nc")]
    
    energy_rows_loc    = []
    power_rows_loc     = []
    discharge_rows_loc = []
    
    for fn in nc_files:
        pfad = os.path.join(net_folder, fn)
        jahr = parse_year_from_filename(fn)
        year_label = jahr if jahr is not None else fn
        
        net = pypsa.Network(pfad)
        metrics = extract_metrics_from_network(net, scenario_name, year_label)
        
        energy_rows_loc   .extend(metrics["energy"])
        power_rows_loc    .extend(metrics["power"])
        discharge_rows_loc.extend(metrics["discharge"])
        
        del net
        gc.collect()
    
    df_energy_loc    = pd.DataFrame(energy_rows_loc)
    df_power_loc     = pd.DataFrame(power_rows_loc)
    df_discharge_loc = pd.DataFrame(discharge_rows_loc)
    
    # Sortierung analog
    sort_cols = ["Scenario", "Type", "Year"]
    df_energy_loc    = df_energy_loc   .sort_values(sort_cols).reset_index(drop=True)
    df_power_loc     = df_power_loc    .sort_values(sort_cols).reset_index(drop=True)
    df_discharge_loc = df_discharge_loc.sort_values(sort_cols).reset_index(drop=True)
    
    return df_energy_loc, df_power_loc, df_discharge_loc

# 3) Rufe load_single_scenario für das neue Szenario auf:
df_energy_new, df_power_new, df_discharge_new = load_single_scenario(base_folder, new_scenario)

# 4) Hänge an die existierenden DataFrames an:
df_energy    = pd.concat([df_energy,    df_energy_new],    ignore_index=True)
df_power     = pd.concat([df_power,     df_power_new],     ignore_index=True)
df_discharge = pd.concat([df_discharge, df_discharge_new], ignore_index=True)

# 5) (Optional) Sortiere wieder, um alles sauber geordnet zu haben:
df_energy    = df_energy   .sort_values(["Scenario","Type","Year"]).reset_index(drop=True)
df_power     = df_power    .sort_values(["Scenario","Type","Year"]).reset_index(drop=True)
df_discharge = df_discharge.sort_values(["Scenario","Type","Year"]).reset_index(drop=True)


In [11]:
print("=== df_energy  ===", df_energy.shape)
print(df_energy.head(), "\n")
print("=== df_power   ===", df_power.shape)
print(df_power.head(), "\n")
print("=== df_discharge ===", df_discharge.shape)
print(df_discharge.head(), "\n")
print("=== df_bev_range ===", df_bev_range.shape)
print(df_bev_range.head())


=== df_energy  === (105, 4)
                 Scenario  Year        Type  Value [GWh]
0  KN2045_Mix_1H_DE_1ZONE  2020  EV battery     0.000000
1  KN2045_Mix_1H_DE_1ZONE  2025  EV battery     0.000000
2  KN2045_Mix_1H_DE_1ZONE  2030  EV battery   357.869828
3  KN2045_Mix_1H_DE_1ZONE  2035  EV battery   536.804741
4  KN2045_Mix_1H_DE_1ZONE  2040  EV battery   835.029598 

=== df_power   === (105, 4)
                 Scenario  Year        Type  Value [GW]
0  KN2045_Mix_1H_DE_1ZONE  2020  EV battery   26.243787
1  KN2045_Mix_1H_DE_1ZONE  2025  EV battery   78.731362
2  KN2045_Mix_1H_DE_1ZONE  2030  EV battery  157.462724
3  KN2045_Mix_1H_DE_1ZONE  2035  EV battery  236.194086
4  KN2045_Mix_1H_DE_1ZONE  2040  EV battery  367.413023 

=== df_discharge === (70, 4)
                 Scenario  Year                Type  Value [TWh]
0  KN2045_Mix_1H_DE_1ZONE  2020  battery discharger     0.000035
1  KN2045_Mix_1H_DE_1ZONE  2025  battery discharger     0.000030
2  KN2045_Mix_1H_DE_1ZONE  2030  batte

In [15]:
# 1) Mapping-Dict anlegen
rename_dict = {
    'Technologiemix': 'Ariadne Report Technologiemix Szenario',
    'KN2045_Mix_3H': 'Eigener Technologiemix Szenario Run',
    'KN2045_Mix_3H_DE_1ZONE': '1-Zone (DE)',
    'KN2045_Mix_1H_DE_1ZONE': '1H Res, 1-Zone (DE)',
    'KN2045_Mix_Base_home_battery': '1H Res, 1-Zone (DE), Min. Home Battery',
    # … alle weiteren Zuordnungen hier
}

# 2) Auf alle DataFrames anwenden
for df in [df_energy, df_power, df_discharge, df_bev_range]:
    df['Scenario'] = df['Scenario'].replace(rename_dict)

# 3) Kontrollieren
print(df_energy['Scenario'].unique())

['KN2045_Mix_1H_DE_1ZONE', 'KN2045_Mix_3H', 'KN2045_Mix_3H_DE_1ZONE', 'KN2045_Mix_Base_home_battery', 'Technologiemix']

['1H Res, 1-Zone (DE)' 'Eigener Technologiemix Szenario Run' '1-Zone (DE)'
 '1H Res, 1-Zone (DE), Min. Home Battery'
 'Ariadne Report Technologiemix Szenario']


['KN2045_Mix_1H_DE_1ZONE',
 'KN2045_Mix_3H',
 'KN2045_Mix_3H_DE_1ZONE',
 'KN2045_Mix_Base_home_battery',
 'Technologiemix']

In [16]:

"""
# ---------------------------------------------------------
# 1. Plot: Entladeleistung (GW) je Speichertyp über die Jahre
# ---------------------------------------------------------
# df_power enthält bereits die Spalten: ["Scenario", "Year", "Type", "Value [GW]"]

fig_gw = px.line(
    df_power,
    x="Year",
    y="Value [GW]",
    color="Type",
    title="Entladeleistung von Batteriespeichern in Deutschland (GW)",
    markers=True
)
fig_gw.update_layout(
    xaxis_title="Jahr",
    yaxis_title="Entladeleistung [GW]"
)
fig_gw.show()



# ---------------------------------------------------------
# 2. Plot: Energiespeicherkapazität (GWh) je Speichertyp über die Jahre
# ---------------------------------------------------------
# df_energy enthält bereits die Spalten: ["Scenario", "Year", "Type", "Value [GWh]"]

fig_gwh = px.line(
    df_energy,
    x="Year",
    y="Value [GWh]",
    color="Type",
    title="Energiespeicherkapazität von Batteriespeichern in Deutschland (GWh)",
    markers=True
)
fig_gwh.update_layout(
    xaxis_title="Jahr",
    yaxis_title="Kapazität [GWh]"
)
fig_gwh.show()



# ---------------------------------------------------------
# 3. Plot: Jährliche Strombereitstellung durch Batterietechnologien (TWh)
# ---------------------------------------------------------
# df_discharge enthält Spalten: ["Scenario", "Year", "Type", "Value [TWh]"],
# wobei "Type" hier die Carrier-Namen ist (z. B. "battery discharger", "home battery discharger", "V2G").
# Wir mappen sie auf die lesbaren Labels:
carrier_map = {
    "battery discharger":        "Utility-scale battery",
    "home battery discharger":   "Home battery",
    "V2G":                       "EV battery"
}

df_discharge_plot = df_discharge.copy()
df_discharge_plot["Type"] = df_discharge_plot["Type"].map(carrier_map)

fig_discharge = px.bar(
    df_discharge_plot,
    x="Year",
    y="Value [TWh]",
    color="Type",
    barmode="group",
    title="Jährliche Strombereitstellung durch Batterietechnologien (TWh)"
)
fig_discharge.update_layout(
    xaxis_title="Jahr",
    yaxis_title="Bereitstellung [TWh]"
)
fig_discharge.show()
"""

'\n# ---------------------------------------------------------\n# 1. Plot: Entladeleistung (GW) je Speichertyp über die Jahre\n# ---------------------------------------------------------\n# df_power enthält bereits die Spalten: ["Scenario", "Year", "Type", "Value [GW]"]\n\nfig_gw = px.line(\n    df_power,\n    x="Year",\n    y="Value [GW]",\n    color="Type",\n    title="Entladeleistung von Batteriespeichern in Deutschland (GW)",\n    markers=True\n)\nfig_gw.update_layout(\n    xaxis_title="Jahr",\n    yaxis_title="Entladeleistung [GW]"\n)\nfig_gw.show()\n\n\n\n# ---------------------------------------------------------\n# 2. Plot: Energiespeicherkapazität (GWh) je Speichertyp über die Jahre\n# ---------------------------------------------------------\n# df_energy enthält bereits die Spalten: ["Scenario", "Year", "Type", "Value [GWh]"]\n\nfig_gwh = px.line(\n    df_energy,\n    x="Year",\n    y="Value [GWh]",\n    color="Type",\n    title="Energiespeicherkapazität von Batteriespeichern

In [22]:
import plotly.express as px

# -----------------------------------------------------------------------------
# 1. Plot: Entladeleistung (GW) je Speichertyp und je Szenario über die Jahre
# -----------------------------------------------------------------------------
#
# Wir nehmen df_power, das die Spalten ["Scenario", "Year", "Type", "Value [GW]"] enthält.

fig_gw = px.line(
    df_power,
    x="Year",
    y="Value [GW]",
    color="Type",
    facet_col="Scenario",
    facet_col_wrap=3,  # Bei 3 Szenarien erhältst du eine Zeile mit 3 Spalten
    title="Entladeleistung von Batteriespeichern in Deutschland (GW) nach Szenario",
    markers=True
)

# Beschriftungen anpassen
#fig_gw.update_layout(
#    xaxis_title="Jahr",
#    yaxis_title="Entladeleistung [GW]",
#    legend_title_text="Speichertyp"
#)

# Die einzelnen Facet-Titel etwas kürzer gestalten (entfernt das "Scenario=")
# Falls die automatische Beschriftung "Scenario=Technologiemix" etc. lautet, kürzen wir sie:
for anno in fig_gw.layout.annotations:
    # annotation text sieht typischerweise aus wie "Scenario=Technologiemix"
    parts = anno.text.split("=")
    if len(parts) == 2:
        anno.text = parts[1]

fig_gw.show()



# -----------------------------------------------------------------------------
# 2. Plot: Energiespeicherkapazität (GWh) je Speichertyp und je Szenario
# -----------------------------------------------------------------------------
#
# Wir nehmen df_energy, das die Spalten ["Scenario", "Year", "Type", "Value [GWh]"] enthält.

fig_gwh = px.line(
    df_energy,
    x="Year",
    y="Value [GWh]",
    color="Type",
    facet_col="Scenario",
    facet_col_wrap=3,
    title="Energiespeicherkapazität von Batteriespeichern in Deutschland (GWh) nach Szenario",
    markers=True
)

#fig_gwh.update_layout(
#    xaxis_title="Jahr",
#    yaxis_title="Kapazität [GWh]",
#    legend_title_text="Speichertyp"
#)

for anno in fig_gwh.layout.annotations:
    parts = anno.text.split("=")
    if len(parts) == 2:
        anno.text = parts[1]

fig_gwh.show()



# -----------------------------------------------------------------------------
# 3. Plot: Jährliche Strombereitstellung durch Batterietechnologien (TWh)
#    – je Typ, je Szenario
# -----------------------------------------------------------------------------
#
# df_discharge enthält ["Scenario", "Year", "Type", "Value [TWh]"].
# Wir mappen die Carrier-Namen auf lesbare Typ-Bezeichnungen:

carrier_map = {
    "battery discharger":        "Utility-scale battery",
    "home battery discharger":   "Home battery",
    "V2G":                       "EV battery"
}

df_discharge_plot = df_discharge.copy()
df_discharge_plot["Type"] = df_discharge_plot["Type"].map(carrier_map)

fig_discharge = px.bar(
    df_discharge_plot,
    x="Year",
    y="Value [TWh]",
    color="Type",
    facet_col="Scenario",
    facet_col_wrap=3,
    barmode="group",
    title="Jährliche Strombereitstellung durch Batterietechnologien (TWh) nach Szenario"
)

fig_discharge.update_layout(
    xaxis_title="Jahr",
    yaxis_title="Bereitstellung [TWh]",
    legend_title_text="Speichertyp"
)

for anno in fig_discharge.layout.annotations:
    parts = anno.text.split("=")
    if len(parts) == 2:
        anno.text = parts[1]

fig_discharge.show()

# 2. Figuren in diesem Ordner speichern
plots = {
    "entladeleistung split scenario":    fig_gw,
    "kapazitaet split scenario":         fig_gwh,
    "bereitstellung split scenario":     fig_discharge
}

for name, fig in plots.items():
    fig.write_html(
        os.path.join("Plots_MA", f"plot_{name}.html"),
        include_plotlyjs="cdn",
        full_html=True
    )


c:\Users\morit\.conda\envs\pypsa-de\Lib\site-packages\_plotly_utils\basevalidators.py:2596: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



c:\Users\morit\.conda\envs\pypsa-de\Lib\site-packages\_plotly_utils\basevalidators.py:2596: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



c:\Users\morit\.conda\envs\pypsa-de\Lib\site-packages\_plotly_utils\basevalidators.py:2596: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [19]:
import plotly.express as px

# -----------------------------------------------------------------------------
# 1. Kombinierte Plot‐Version: Entladeleistung (GW) je Speichertyp und Szenario
# -----------------------------------------------------------------------------
#
# Hier nutzen wir df_power mit den Spalten ["Scenario", "Year", "Type", "Value [GW]"].
# Wir zeichnen pro (Type, Scenario) eine eigene Linie, indem wir color="Type"
# beibehalten und zusätzlich line_dash="Scenario" setzen.

fig_gw_all = px.line(
    df_power,
    x="Year",
    y="Value [GW]",
    color="Type",          # gleiche Farben für jeden Speichertyp
    line_dash="Scenario",  # unterschiedliche Linientypen (Strichmuster) für jedes Szenario
    title="Entladeleistung von Batteriespeichern in Deutschland (GW) – alle Szenarien",
    markers=True
)

fig_gw_all.update_layout(
    xaxis_title="Jahr",
    yaxis_title="Entladeleistung [GW]",
    legend_title_text="Speichertyp / Szenario"
)

fig_gw_all.show()



# -----------------------------------------------------------------------------
# 2. Kombinierte Plot‐Version: Energiespeicherkapazität (GWh) je Speichertyp und Szenario
# -----------------------------------------------------------------------------
#
# Hier nutzen wir df_energy mit den Spalten ["Scenario", "Year", "Type", "Value [GWh]"].
# Auch hier farblich nach Speichertyp, dash‐Linienmuster nach Szenario.

fig_gwh_all = px.line(
    df_energy,
    x="Year",
    y="Value [GWh]",
    color="Type",
    line_dash="Scenario",
    title="Energiespeicherkapazität von Batteriespeichern in Deutschland (GWh) – alle Szenarien",
    markers=True
)

fig_gwh_all.update_layout(
    xaxis_title="Jahr",
    yaxis_title="Kapazität [GWh]",
    legend_title_text="Speichertyp / Szenario"
)

fig_gwh_all.show()



# -----------------------------------------------------------------------------
# 3. Kombinierte Plot‐Version: Jährliche Strombereitstellung (TWh) je Speichertyp und Szenario
# -----------------------------------------------------------------------------
#
# Für df_discharge (["Scenario","Year","Type","Value [TWh]"]) mappen wir zunächst
# die Carrier‐Namen auf lesbare Speichertypen, dann nutzen wir color="Type"
# und line_dash="Scenario". Da es sich um eine Bar‐Chart handelt, verwenden wir
# allerdings px.bar(). Leider kennt px.bar() kein line_dash, deshalb erstellen wir hier
# ein umgebendes Facet für Scenario und color für Type, alle in einem Bild:
#
#    – color="Type" unterscheidet die Balkenfarben nach Speichertyp
#    – pattern_shape="Scenario" („Füllmuster“) trennt zusätzlich die Szenarien;
#      Plotly Express unterstützt pattern_shape mittlerweile auch bei bar‐Plots.
#
# Wenn dein Plotly-Upstream diese Version noch nicht unterstützt, kannst du alternativ
# alle Szenarien mit opacity (Transparenz) überlagern oder auf eine reine Linien‐Variante
# (px.line mit statistics) zurückgreifen – im Folgenden zeige ich die pattern_shape‐Variante.

carrier_map = {
    "battery discharger":      "Utility-scale battery",
    "home battery discharger": "Home battery",
    "V2G":                     "EV battery"
}

df_discharge_plot = df_discharge.copy()
df_discharge_plot["Type"] = df_discharge_plot["Type"].map(carrier_map)

# px.bar mit pattern_shape
fig_discharge_all = px.bar(
    df_discharge_plot,
    x="Year",
    y="Value [TWh]",
    color="Type",
    pattern_shape="Scenario",
    barmode="group",
    title="Jährliche Strombereitstellung durch Batterietechnologien (TWh) – alle Szenarien"
)

fig_discharge_all.update_layout(
    xaxis_title="Jahr",
    yaxis_title="Bereitstellung [TWh]",
    legend_title_text="Speichertyp / Szenario"
)

fig_discharge_all.show()

# 2. Figuren in diesem Ordner speichern
plots = {
    "entladeleistung":    fig_gw_all,
    "kapazitaet":         fig_gwh_all,
    "bereitstellung":     fig_discharge_all
}

for name, fig in plots.items():
    fig.write_html(
        os.path.join("Plots_MA", f"plot_{name}.html"),
        include_plotlyjs="cdn",
        full_html=True
    )

c:\Users\morit\.conda\envs\pypsa-de\Lib\site-packages\_plotly_utils\basevalidators.py:2596: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



c:\Users\morit\.conda\envs\pypsa-de\Lib\site-packages\_plotly_utils\basevalidators.py:2596: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



c:\Users\morit\.conda\envs\pypsa-de\Lib\site-packages\_plotly_utils\basevalidators.py:2596: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [41]:
fig_bev = go.Figure()

# a) BEV-Charger-Spannweite für jedes Szenario zeichnen
for scenario in df_bev_range["Scenario"].unique():
    df_scen_bev = df_bev_range[df_bev_range["Scenario"] == scenario]
    df_min  = df_scen_bev[df_scen_bev["Type"] == "EV battery (BEV charger) - Min"].sort_values("Year")
    df_mean = df_scen_bev[df_scen_bev["Type"] == "EV battery (BEV charger) - Mean"].sort_values("Year")
    df_max  = df_scen_bev[df_scen_bev["Type"] == "EV battery (BEV charger) - Max"].sort_values("Year")

    x = df_mean["Year"]

    # Max-Linie
    fig_bev.add_trace(go.Scatter(
        x=x,
        y=df_max["Value [GW]"],
        mode="lines+markers",
        line=dict(
            color=bev_colors["EV battery (BEV charger) - Max"],
            dash=bev_dash["EV battery (BEV charger) - Max"]
        ),
        name=f"{scenario}: BEV charger – Max"
    ))

    # Fläche zwischen Min und Max (schattierte Spannweite)
    fig_bev.add_trace(go.Scatter(
        x=x.tolist() + x[::-1].tolist(),
        y=(df_max["Value [GW]"].tolist() + df_min["Value [GW]"][::-1].tolist()),
        fill="toself",
        fillcolor="rgba(0, 180, 255, 0.2)",
        line=dict(color="rgba(255,255,255,0)"),
        hoverinfo="skip",
        showlegend=False,
        name=f"{scenario}: BEV charger – Range"
    ))

    # Min-Linie
    fig_bev.add_trace(go.Scatter(
        x=x,
        y=df_min["Value [GW]"],
        mode="lines+markers",
        line=dict(
            color=bev_colors["EV battery (BEV charger) - Min"],
            dash=bev_dash["EV battery (BEV charger) - Min"]
        ),
        name=f"{scenario}: BEV charger – Min"
    ))

    # Mean-Linie
    fig_bev.add_trace(go.Scatter(
        x=x,
        y=df_mean["Value [GW]"],
        mode="lines+markers",
        line=dict(color=bev_colors["EV battery (BEV charger) - Mean"], width=2),
        name=f"{scenario}: BEV charger – Mean"
    ))

# b) Haupt‐Speicherkurven (Utility-scale, Home, EV battery) pro Szenario
main_techs = ["Utility-scale battery", "Home battery", "EV battery"]
for tech in main_techs:
    df_tech = df_power[df_power["Type"] == tech]
    for scenario in df_tech["Scenario"].unique():
        df_sub = df_tech[df_tech["Scenario"] == scenario].sort_values("Year")
        if df_sub.empty:
            continue
        fig_bev.add_trace(go.Scatter(
            x=df_sub["Year"],
            y=df_sub["Value [GW]"],
            mode="lines+markers",
            name=f"{scenario}: {tech}"
        ))

# c) Layout anpassen
fig_bev.update_layout(
    title="Effektiv verfügbare Ladeleistung von Batteriespeichern (GW) – alle Szenarien inkl. BEV-Charger",
    xaxis_title="Jahr",
    yaxis_title="Verfügbare Ladeleistung [GW]",
    template="plotly_white",
    legend_title_text="Szenario / Typ",
    legend=dict(
        orientation="v",
        yanchor="top",
        y=0.95,
        xanchor="left",
        x=1.02,
        traceorder="normal",
        itemsizing="constant"
    )
)

fig_bev.show()


c:\Users\morit\.conda\envs\pypsa-de\Lib\site-packages\_plotly_utils\basevalidators.py:2596: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [52]:
def get_ev_battery_available_energy_range(net):
    ev_idx = net.stores.index[
        (net.stores.carrier == "EV battery") &
        (net.stores.bus.str.startswith("DE"))
    ]
    if len(ev_idx) == 0:
        return 0.0, 0.0, 0.0

    e_nom = net.stores.loc[ev_idx, "e_nom"]
    e_min_pu = net.stores_t.e_min_pu[ev_idx]
    e_max_pu = 1  # am besten nicht hard gecoded falls sich value ändert

    print("Max min_pu:", e_min_pu.max().max())
    print("Min min_pu:", e_min_pu.min().min())

    e_avail = (e_max_pu - e_min_pu).multiply(e_nom, axis=1)

    print("Min verfügbar:", e_avail.sum(axis=1).min())

    total_per_snapshot = e_avail.sum(axis=1)
    return (
        total_per_snapshot.min() / 1e3,
        total_per_snapshot.mean() / 1e3,
        total_per_snapshot.max() / 1e3,
    )

get_ev_battery_available_energy_range(n[2040])

ev_energy_range = []

for year in years:
    net = n[year]
    e_min, e_mean, e_max = get_ev_battery_available_energy_range(net)
    ev_energy_range.append({"Year": year, "Type": "EV battery available - Min", "Value [GWh]": e_min})
    ev_energy_range.append({"Year": year, "Type": "EV battery available - Mean", "Value [GWh]": e_mean})
    ev_energy_range.append({"Year": year, "Type": "EV battery available - Max", "Value [GWh]": e_max})

df_ev_energy_range = pd.DataFrame(ev_energy_range)
df_energy_all = pd.concat([df_energy, df_ev_energy_range])

import plotly.graph_objects as go

# Daten für EV battery available
df_min = df_energy_all[df_energy_all["Type"] == "EV battery available - Min"]
df_mean = df_energy_all[df_energy_all["Type"] == "EV battery available - Mean"]
df_max = df_energy_all[df_energy_all["Type"] == "EV battery available - Max"]
x = df_mean["Year"]

fig = go.Figure()

# Max-Linie (sichtbar)
fig.add_trace(go.Scatter(
    x=x,
    y=df_max["Value [GWh]"],
    mode="lines+markers",
    line=dict(color="lightblue", dash="dot"),
    name="EV battery available - Max"
))

# Fläche zur Max-Linie von unten (tonexty)
fig.add_trace(go.Scatter(
    x=x,
    y=df_min["Value [GWh]"],
    mode="lines",
    fill="tonexty",
    fillcolor="rgba(0, 100, 255, 0.2)",
    line=dict(width=0),
    name="Spannweite",
    hoverinfo="skip",
    showlegend=False
))

# Min-Linie (sichtbar)
fig.add_trace(go.Scatter(
    x=x,
    y=df_min["Value [GWh]"],
    mode="lines+markers",
    line=dict(color="lightblue", dash="dot"),
    name="EV battery available - Min"
))

# Mittelwertlinie (sichtbar)
fig.add_trace(go.Scatter(
    x=x,
    y=df_mean["Value [GWh]"],
    mode="lines+markers",
    line=dict(color="blue", width=2),
    name="EV battery available - Mean"
))

# Weitere Speicherarten (aus df_energy_all)
for tech in ["Utility-scale battery", "Home battery", "EV battery"]:
    df_ = df_energy_all[df_energy_all["Type"] == tech]
    fig.add_trace(go.Scatter(
        x=df_["Year"],
        y=df_["Value [GWh]"],
        mode="lines+markers",
        name=tech
    ))

fig.update_layout(
    title="Nutzbare EV-Speicherkapazität mit e_min_pu (GWh)",
    xaxis_title="Jahr",
    yaxis_title="Verfügbare Kapazität [GWh]",
    template="plotly_white"
)

fig.show()


NameError: name 'n' is not defined

In [34]:
import xarray as xr
import pandas as pd
import plotly.graph_objects as go
from pathlib import Path

# Szenarien und Labels
scenarios = {
    'KN2045_Mix_3H_DE_1ZONE': '1-Zone (DE)',
    'KN2045_Mix_1H_DE_1ZONE': '1H Res, 1-Zone (DE)'
}

# Analyse-Jahr (z.B. 2045) und Monate
analysis_year = '2045'
months = {
    f'{analysis_year}-08': 'August',
    f'{analysis_year}-01': 'Januar'
}

# Durch alle Szenarien iterieren
for month_key, month_label in months.items():
    fig = go.Figure()
    for scenario_id, scenario_label in scenarios.items():
        net_folder = Path('results') / scenario_id / 'networks'
        # Nur NetCDF-Dateien für das Analyse-Jahr auswählen
        nc_files = sorted(net_folder.glob(f'*{analysis_year}*.nc'))
        if not nc_files:
            print(f"Keine Dateien für Jahr {analysis_year} in Szenario {scenario_id}")
            continue

        nc_file = nc_files[0]  # Ein Datei pro Szenario
        try:
            ds = xr.open_dataset(nc_file)
        except Exception as e:
            print(f"Fehler beim Laden {nc_file.name}: {e}")
            continue

        # Zeitstempel
        if 'snapshots' in ds.coords:
            times = pd.DatetimeIndex(ds['snapshots'].values)
        else:
            coord0 = list(ds.coords)[0]
            times = pd.DatetimeIndex(ds.coords[coord0].values)
        mask = times.strftime('%Y-%m') == month_key

        # Link-Dispatch Variable bestimmen
        if 'links_t_p0' in ds.data_vars:
            dispatch = ds['links_t_p0']
        elif 'links_t_p1' in ds.data_vars:
            dispatch = ds['links_t_p1']
        else:
            print(f"Keine Link-Dispatch-Variable in {nc_file.name}")
            continue

        # Link-Dimension finden
        link_dim = next((d for d in dispatch.dims if 'link' in d), None)
        if link_dim is None:
            print(f"Keine Link-Dimension in {nc_file.name}")
            continue

        # Discharger-Links filtern
        available_links = list(dispatch.coords[link_dim].values)
        dischargers = [l for l in available_links if 'discharger' in str(l).lower()]
        if not dischargers:
            print(f"Keine Discharger-Links in {nc_file.name}")
            continue

        # Für jeden Discharger die Entladungen plotten
        for link_id in dischargers:
            data = dispatch.sel({link_dim: link_id}).values
            # Entladung: negative Fluss ⇒ positiv darstellen
            discharge = pd.Series(-data, index=times).clip(lower=0)
            discharge_month = discharge[mask]
            fig.add_trace(
                go.Scatter(
                    x=discharge_month.index,
                    y=discharge_month.values,
                    mode='lines',
                    name=f"{link_id} – {scenario_label}"
                )
            )

    fig.update_layout(
        title=f"Stündliche Entladungen der Batterie-Discharger im {month_label} {analysis_year}",
        xaxis_title="Zeit",
        yaxis_title="Entladungsleistung (MW)",
        legend_title="Szenario & Link",
        hovermode='x unified'
    )
    fig.show()


In [37]:
import xarray as xr
import pandas as pd
import plotly.graph_objects as go
from pathlib import Path

# Szenarien und Labels
scenarios = {
    'KN2045_Mix_3H_DE_1ZONE': '1-Zone (DE)',   # 3H-Szenario
    'KN2045_Mix_1H_DE_1ZONE': '1H Res, 1-Zone (DE)'  # 1H-Szenario
}

# Analysejahr und Monate
analysis_year = '2045'
months = {
    f'{analysis_year}-08': 'August',
    f'{analysis_year}-01': 'Januar'
}

# Durch Szenarien und Monate iterieren
for month_key, month_label in months.items():
    fig = go.Figure()
    for scenario_id, scenario_label in scenarios.items():
        net_folder = Path('results') / scenario_id / 'networks'
        # Alle .nc-Dateien des Jahres
        all_nc = list(net_folder.glob(f'*{analysis_year}*.nc'))
        if not all_nc:
            print(f"Keine {analysis_year}-Dateien in {scenario_id}")
            continue
        # Nach Auflösung filtern
        if '1H' in scenario_id:
            # 1H-Szenario: nur Dateien mit 'hourly' im Namen
            candidates = [f for f in all_nc if 'hourly' in f.name.lower()]
        else:
            # 3H-Szenario: nur Dateien ohne 'hourly'
            candidates = [f for f in all_nc if 'hourly' not in f.name.lower()]
        if not candidates:
            print(f"Keine passenden Dateien für {scenario_label} im Jahr {analysis_year}")
            continue
        nc_file = sorted(candidates)[0]
        print(f"Lade {nc_file.name} für Szenario {scenario_label}")

        # NetCDF laden
        try:
            ds = xr.open_dataset(nc_file)
        except Exception as e:
            print(f"Fehler beim Laden {nc_file.name}: {e}")
            continue

        # Zeitstempel
        coord = 'snapshots' if 'snapshots' in ds.coords else list(ds.coords)[0]
        times = pd.DatetimeIndex(ds.coords[coord].values)
        mask = times.strftime('%Y-%m') == month_key

        # Link-Dispatch Variable
        if 'links_t_p0' in ds.data_vars:
            dispatch = ds['links_t_p0']
        elif 'links_t_p1' in ds.data_vars:
            dispatch = ds['links_t_p1']
        else:
            print(f"Keine Link-Dispatch-Variable in {nc_file.name}")
            continue

        # Link-Dimension
        link_dim = next((d for d in dispatch.dims if 'link' in d), None)
        if link_dim is None:
            print(f"Keine Link-Dimension in {nc_file.name}")
            continue

        # Discharger-Links
        links = list(dispatch.coords[link_dim].values)
        dischargers = [l for l in links if 'discharger' in str(l).lower()]
        if not dischargers:
            print(f"Keine Discharger-Links in {nc_file.name}")
            continue

        # Plotten
        for link_id in dischargers:
            data = dispatch.sel({link_dim: link_id}).values
            # Entladung: negativer Fluss ⇒ positiv
            discharge = pd.Series(-data, index=times).clip(lower=0)
            discharge_month = discharge[mask]
            fig.add_trace(
                go.Scatter(
                    x=discharge_month.index,
                    y=discharge_month.values,
                    mode='lines',
                    name=f"{scenario_label} – {link_id}"
                )
            )

    fig.update_layout(
        title=f"Stündliche Batteriedischargen im {month_label} {analysis_year}",
        xaxis_title="Zeit",
        yaxis_title="Entladungsleistung (MW)",
        legend_title="Szenario – Link",
        hovermode='x unified'
    )
    fig.show()


Lade base_s_20_RedSpa_none_2045.nc für Szenario 1-Zone (DE)
Lade base_s_20_hourly_none_2045.nc für Szenario 1H Res, 1-Zone (DE)


Lade base_s_20_RedSpa_none_2045.nc für Szenario 1-Zone (DE)
Lade base_s_20_hourly_none_2045.nc für Szenario 1H Res, 1-Zone (DE)
